**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](manipulation.csail.mit.edu/pick.html).**

# Notebook setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- launch a server for our 3D visualizer (MeshCat) that will be used for the remainder of this notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake.
if 'google.colab' in sys.modules and importlib.util.find_spec('pydrake') is None:
  version='20200907'
  build='nightly'
  urlretrieve(f"https://drake-packages.csail.mit.edu/drake/{build}/drake-{version}/setup_drake_colab.py",
              "setup_drake_colab.py")
  from setup_drake_colab import setup_drake
  setup_drake(version="20200907190552-4413ec877e8c1c8237023d74355b76c391a11ca2", build="experimental")
#  setup_drake(version=version, build=build)

# Install pyngrok.
server_args = []
if 'google.colab' in sys.modules:
  !pip install pyngrok
  server_args = ['--ngrok_http_tunnel']

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)


# TODO(russt): upstream this to drake
import numpy as np
from functools import partial

from IPython.display import display
from ipywidgets import FloatSlider, Layout

from pydrake.all import JointIndex

slider_widgets = []
def joint_sliders_with_callbacks(
    robot, meshcat, root_context, extra_callback=None, 
    lower_limit=-10., upper_limit=10., resolution=0.01, length=200):
    """ will add a slider for each joint in plant, and set up a callback to set the context and call meshcat publish. """

    def _reshape(x, num):
        x = np.array(x)
        assert len(x.shape) <= 1
        return np.array(x) * np.ones(num)

    lower_limit = _reshape(lower_limit, robot.num_positions())
    upper_limit = _reshape(upper_limit, robot.num_positions())
    resolution = _reshape(resolution, robot.num_positions())

    meshcat.load()
    meshcat_context = meshcat.GetMyContextFromRoot(root_context)
    meshcat.Publish(meshcat_context)
    robot_context = robot.GetMyContextFromRoot(root_context)
    positions = robot.GetPositions(robot_context)
    def _slider_callback(change, index):
        positions[index] = change.new
        robot.SetPositions(robot_context, positions)
        meshcat.Publish(meshcat_context)
        if extra_callback:
            extra_callback(robot_context)

    k = 0
    for i in range(0, robot.num_joints()):
        joint = robot.get_joint(JointIndex(i))
        low = joint.position_lower_limits()
        upp = joint.position_upper_limits()
        for j in range(0, joint.num_positions()):
            index = joint.position_start() + j
            slider = FloatSlider(
                value=positions[index],
                min=max(low[j], lower_limit[k]),
                max=min(upp[j], upper_limit[k]),
                step=resolution[k],
                continuous_update=True,
                description=joint.name(),
                style={'description_width': 'initial'},
                layout=Layout(width=f"'{length}'"))
            slider.observe(partial(_slider_callback, index=index), names='value')
            display(slider)
            slider_widgets.append(slider)
            k += 1

def close_sliders():
    for w in slider_widgets:
        w.close()


# TODO(russt): Move this to drake (adding the element name support to the base class).
import pandas as pd

def dataframe(trajectory, times, names):
  assert trajectory.rows() == len(names)
  values = trajectory.vector_values(times)
  data = {'t': times }
  for i in range(len(names)):
    data[names[i]] = values[i,:]
  return pd.DataFrame(data)

# Inspecting the kinematic tree

Here is a simple example that demonstrates how to inspect the kinematic tree stored in a `MultibodyPlant`.

In [ ]:
import pydot
from IPython.display import SVG
from pydrake.all import FindResourceOrThrow, MultibodyPlant, Parser

plant = MultibodyPlant(time_step=0.0)
parser = Parser(plant)
parser.AddModelFromFile(FindResourceOrThrow(
    "drake/manipulation/models/allegro_hand_description/sdf/allegro_hand_description_right.sdf"))
parser.AddModelFromFile(FindResourceOrThrow(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf"))
plant.Finalize()

# TODO(russt): Add floating base connections
# TODO(russt): Consider a more interactive javascript rendering?
SVG(pydot.graph_from_dot_data(plant.GetTopologyGraphvizString())[0].create_svg())

# Forward kinematics of the gripper frame

Here is a simple example that let's you visualize the frames on the iiwa and the gripper.  If you click on the "Open Controls" menu in the MeshCat visualizer, and dig into the menu `meshcat->drake->Source` then you will see elements for each of the models in the `SceneGraph`: one for the iiwa, another for the WSG, and others for the clutter bins.  You can enable/disable their visualization.  Give it a spin!

In [ ]:
from IPython.display import display
from ipywidgets import Text, Layout

from pydrake.all import DiagramBuilder, ConnectMeshcatVisualizer, RollPitchYaw
from pydrake.examples.manipulation_station import ManipulationStation

close_sliders()
builder = DiagramBuilder()

# TODO: Replace this with a simple model directive of iiwa+wsg (no clutter bins)
station = builder.AddSystem(ManipulationStation())
station.SetupClutterClearingStation()
station.Finalize()

frames_to_draw = {"iiwa": {"iiwa_link_1", "iiwa_link_2", "iiwa_link_3", "iiwa_link_4", "iiwa_link_5", "iiwa_link_6", "iiwa_link_7"},
                  "gripper": {"body"}}
meshcat = ConnectMeshcatVisualizer(builder,
    station.get_scene_graph(),
    output_port=station.GetOutputPort("pose_bundle"),
    zmq_url=zmq_url,
    frames_to_draw=frames_to_draw,
    axis_length=0.3,
    axis_radius=0.01)

diagram = builder.Build()
context = diagram.CreateDefaultContext()

xyz = Text(value="", description="gripper position (m): ", layout=Layout(width='500px'), style={'description_width':'initial'})
rpy = Text(value="", description="gripper roll-pitch-yaw (rad): ", layout=Layout(width='500px'), style={'description_width':'initial'})
plant = station.get_multibody_plant()

gripper = plant.GetBodyByName("body")
def pose_callback(context):
    pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
    xyz.value = np.array2string(pose.translation(), formatter={'float': lambda x: "{:3.2f}".format(x)})
    rpy.value = np.array2string(RollPitchYaw(pose.rotation()).vector(), formatter={'float': lambda x: "{:3.2f}".format(x)})

joint_sliders_with_callbacks(station.get_multibody_plant(), meshcat, context, extra_callback=pose_callback)
display(xyz)
display(rpy)


# Don't assume $\dot{q} \equiv v$

Let's just add a single object into the scene.  We won't weld it to the world frame, so it is a "free body" or has a "floating base".

In [ ]:
from pydrake.all import FindResourceOrThrow, MultibodyPlant, Parser

plant = MultibodyPlant(time_step = 0.0)
Parser(plant).AddModelFromFile(FindResourceOrThrow(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf"))
plant.Finalize()

context = plant.CreateDefaultContext()
print(context)

print(f"plant.num_positions() = {plant.num_positions()}")
print(f"plant.num_velocities() = {plant.num_velocities()}")

Looking at the `Context` you can see that this system has 13 total state variables.  7 of them are positions, $q$; this is due to our pose representation using unit quaternions.  But only 6 of them are velocities, $v$; this is because a six-element spatial velocity provides a better (unconstrained) representation of the rate of change of the unit quaternion.  But clearly, if the length of the vectors don't even match, we do *not* have $\dot{q} = v$.

# Kinematic Jacobians for pick and place

Let's set up the same iiwa + wsg example, with sliders (but without the frames), that we used above.  But this time I'll display the value of the Jacobian $J^G(q)$.

In [ ]:
from IPython.display import display
from ipywidgets import Text, Textarea, Layout

from pydrake.all import (DiagramBuilder, ConnectMeshcatVisualizer, JacobianWrtVariable)
from pydrake.examples.manipulation_station import ManipulationStation

close_sliders()
builder = DiagramBuilder()

# TODO: Replace this with a simple model directive of iiwa+wsg (no clutter bins)
station = builder.AddSystem(ManipulationStation())
station.SetupClutterClearingStation()
station.Finalize()

meshcat = ConnectMeshcatVisualizer(builder,
    station.get_scene_graph(),
    output_port=station.GetOutputPort("pose_bundle"),
    zmq_url=zmq_url)

diagram = builder.Build()
context = diagram.CreateDefaultContext()

jacobian = Textarea(value="", description="J_G: ", layout={'width':'60%','height':'100px'}, style={'description_width':'initial'})
sigma = Text(value="", description="smallest singular value(J_G): ", layout={'width':'60%'}, style={'description_width':'initial'})
plant = station.get_multibody_plant()

G = plant.GetBodyByName("body").body_frame()
W = plant.world_frame()
def pose_callback(context):
    J_G = plant.CalcJacobianSpatialVelocity(context, JacobianWrtVariable.kQDot, G, [0,0,0], W, W)   ## This is the important line
    jacobian.value = np.array2string(J_G, formatter={'float': lambda x: "{:5.1f}".format(x)})
    sigma.value = str(np.min(np.linalg.svd(J_G, compute_uv=False)))

joint_sliders_with_callbacks(station.get_multibody_plant(), meshcat, context, extra_callback=pose_callback)
display(jacobian)
display(sigma)


# Our first end-effector "controller"

Let's use the pseudo-inverse of the Jacobian to drive the robot around.  To do that, we'll write a very simple system that looks at the current value of $q$, computes $[J^G]^+$, and uses it to command a constant spatial velocity, $V^G$.

We'll only run this controller for a short duration.  Constant spatial velocities aren't something that makes sense for a longer simulation!

Make sure you try changing $V^G$, and understand how the command relates to the motion of the robot.

In [ ]:
import numpy as np

from pydrake.all import (
    BasicVector, DiagramBuilder, ConnectMeshcatVisualizer, Integrator,
    JacobianWrtVariable, LeafSystem, Simulator)
from pydrake.examples.manipulation_station import ManipulationStation

# We can write a new System by deriving from the LeafSystem class.
# There is a little bit of boiler plate, but hopefully this example makes sense.
class PseudoInverseController(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.DeclareVectorInputPort("iiwa_position", BasicVector(7))
        self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7), 
                                     self.CalcOutput)

    def CalcOutput(self, context, output):
        q = self.get_input_port().Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kQDot, 
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,0:7] # Ignore gripper terms
        
        V_G_desired = np.array([0,    # rotation about x
                                -.1,  # rotation about y
                                0,    # rotation about z
                                0,    # x
                                -.05, # y
                                -.1]) # z
        v = np.linalg.pinv(J_G).dot(V_G_desired)
        output.SetFromVector(v)

        
close_sliders()
builder = DiagramBuilder()

station = builder.AddSystem(ManipulationStation())
station.SetupClutterClearingStation()
station.Finalize()

controller = builder.AddSystem(PseudoInverseController(
    station.get_multibody_plant()))
integrator = builder.AddSystem(Integrator(7))

builder.Connect(controller.get_output_port(), 
                integrator.get_input_port())
builder.Connect(integrator.get_output_port(),
                station.GetInputPort("iiwa_position"))
builder.Connect(station.GetOutputPort("iiwa_position_measured"),
                controller.get_input_port())

meshcat = ConnectMeshcatVisualizer(builder,
    station.get_scene_graph(),
    output_port=station.GetOutputPort("pose_bundle"),
    zmq_url=zmq_url)

diagram = builder.Build()
simulator = Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
station.GetInputPort("iiwa_feedforward_torque").FixValue(station_context, np.zeros((7,1)))
station.GetInputPort("wsg_position").FixValue(station_context, [0.1])
# TODO(russt): Add this missing python binding
#integrator.set_integral_value(
#    integrator.GetMyContextFromRoot(simulator.get_mutable_context()), 
#        station.GetIiwaPosition(station_context))
integrator.GetMyContextFromRoot(simulator.get_mutable_context()).get_mutable_continuous_state_vector().SetFromVector(station.GetIiwaPosition(station_context))

simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(0.01);

In [ ]:
# Wait for the model to load in the visualizer, then run this cell to see the interesting part...
simulator.AdvanceTo(5.0);

# Compute grasp and pregrasp poses

Here is a simple example with a floating Schunk gripper and the foam brick.  It defines the grasp pose as described in the notes, and renders it to the 3D visualizer.

**Check yourself**: Try changing the grasp pose to our pregrasp pose.  Do you like the numbers that I picked in the text?

In [ ]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, ConnectMeshcatVisualizer, 
    FindResourceOrThrow, Parser, RigidTransform, RotationMatrix)

builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step = 0.0)
parser = Parser(plant, scene_graph)
grasp = parser.AddModelFromFile(FindResourceOrThrow(
    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "grasp")
# TODO(russt): Draw the pregrasp gripper, too, as transparent (drake #13970).
#pregrasp = parser.AddModelFromFile(FindResourceOrThrow(
#    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf"), "pregrasp")
brick = parser.AddModelFromFile(FindResourceOrThrow(
    "drake/examples/manipulation_station/models/061_foam_brick.sdf"))
plant.Finalize()


meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

# TODO(russt): Set a random pose of the object.

# Get the current object, O, pose
B_O = plant.GetBodyByName("base_link", brick)
X_WO = plant.EvalBodyPoseInWorld(plant_context, B_O)

B_Ggrasp = plant.GetBodyByName("body", grasp)
p_GgraspO = [0, 0.12, 0]
R_GgraspO = RotationMatrix.MakeXRotation(np.pi/2.0).multiply(
    RotationMatrix.MakeZRotation(np.pi/2.0))
X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
X_OGgrasp = X_GgraspO.inverse()
X_WGgrasp = X_WO.multiply(X_OGgrasp)

plant.SetFreeBodyPose(plant_context, B_Ggrasp, X_WGgrasp)
# Open the fingers, too.
plant.GetJointByName("left_finger_sliding_joint", grasp).set_translation(plant_context, 0.054)
plant.GetJointByName("right_finger_sliding_joint", grasp).set_translation(plant_context, -0.054)

meshcat.load()
diagram.Publish(context)

# Pick and place trajectories

Let's first define the canonical frames of our problem.  We've got a handful of them, and will pass them in and out of several functions, so I'll use a dictionary to store them.

In [ ]:
from pydrake.all import RigidTransform, RotationMatrix

def make_gripper_frames(X_G, X_O):
  """
  Takes a partial specification with X_G["initial"] and X_O["initial"] and X_0["goal"], and 
  returns a X_G and times with all of the pick and place frames populated.
  """
  # Define (again) the gripper pose relative to the object when in grasp.
  p_GgraspO = [0, 0.12, 0]
  R_GgraspO = RotationMatrix.MakeXRotation(np.pi/2.0).multiply(
      RotationMatrix.MakeZRotation(np.pi/2.0))
  X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
  X_OGgrasp = X_GgraspO.inverse()
  # pregrasp is negative y in the gripper frame (see the figure!).
  X_GgraspGpregrasp = RigidTransform([0, -0.08, 0])

  X_G["pick"] = X_O["initial"].multiply(X_OGgrasp)
  X_G["prepick"] = X_G["pick"].multiply(X_GgraspGpregrasp)
  X_G["place"] = X_O["goal"].multiply(X_OGgrasp)
  X_G["preplace"] = X_G["place"].multiply(X_GgraspGpregrasp)

  # Now let's set the timing
  times = {"initial": 0}
  X_GinitialGprepick = X_G["initial"].inverse().multiply(X_G["prepick"])
  times["prepick"] = times["initial"] + 20.0*np.linalg.norm(X_GinitialGprepick.translation())
  # Allow some time for the gripper to close.
  times["pick_start"] = times["prepick"] + 2.0
  times["pick_end"] = times["pick_start"] + 2.0
  times["postpick"] = times["pick_end"] + 2.0
  X_GprepickGpreplace = X_G["prepick"].inverse().multiply(X_G["preplace"])
  times["preplace"] = times["postpick"] + 20.0*np.linalg.norm(X_GprepickGpreplace.translation())
  times["place_start"] = times["preplace"] + 2.0
  times["place_end"] = times["place_start"] + 2.0
  times["postplace"] = times["place_end"] + 2.0

  return X_G, times

X_G = {"initial": RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2.0), [0, -0.25, 0.25])}
X_O = {"initial": RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [-.2, -.25, 0.025]),
       "goal": RigidTransform([.25, 0, 0.025])}
X_G, times = make_gripper_frames(X_G, X_O)
print(f"Sanity check: The entire maneuver will take {times['postplace']} seconds to execute.")

Here's a simple visualization of those frames.  Use the controls menu in meshcat to view the different objects on the different "paths" to understand which objects belong to which frames.  Everything look reasonable?

Try changing the initial and final pose of the gripper / object.

In [ ]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, ConnectMeshcatVisualizer, 
    FindResourceOrThrow, Parser, RigidTransform, RotationMatrix)

def visualize_gripper_frames(X_G, X_O):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step = 0.0)
    parser = Parser(plant, scene_graph)
    gripper = FindResourceOrThrow(
        "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf")
    object = FindResourceOrThrow(
        "drake/examples/manipulation_station/models/061_foam_brick.sdf")
    for key, pose in X_G.items():
      g = parser.AddModelFromFile(gripper, f"gripper_{key}")
      plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("body", g), pose)
    for key, pose in X_O.items():
      o = parser.AddModelFromFile(object, f"object_{key}")
      plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base_link", o), pose)

    plant.Finalize()


    meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url)
    meshcat.load()

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)

visualize_gripper_frames(X_G, X_O)

We've defined the keyframes, but now we will fill in timing details (what should be happening at each instant).  Interpolating between the positions is quite simple; we can encode these details as a `PiecewisePolynomial` trajectory in $\Re^3$ over time.

In [ ]:
from pydrake.all import PiecewisePolynomial

def make_gripper_position_trajectory(X_G, times):
    """
    Constructs a gripper position trajectory from the plan "sketch".
    """
    # The syntax here is a little ugly; we need to pass in a matrix with the samples in the columns.
    # TODO(russt): Add python bindings for the std::vector constructor.
    traj = PiecewisePolynomial.FirstOrderHold(
        [times["initial"], times["prepick"]], np.vstack([X_G["initial"].translation(), X_G["prepick"].translation()]).T)

    # TODO(russt): I could make this less brittle if I was more careful on the names above, and just look up the pose for every time (in order)
    traj.AppendFirstOrderSegment(times["pick_start"], X_G["pick"].translation())
    traj.AppendFirstOrderSegment(times["pick_end"], X_G["pick"].translation())
    traj.AppendFirstOrderSegment(times["postpick"], X_G["prepick"].translation())
    traj.AppendFirstOrderSegment(times["preplace"], X_G["preplace"].translation())
    traj.AppendFirstOrderSegment(times["place_start"], X_G["place"].translation())
    traj.AppendFirstOrderSegment(times["place_end"], X_G["place"].translation())
    traj.AppendFirstOrderSegment(times["postplace"], X_G["preplace"].translation())

    return traj

traj_p_G = make_gripper_position_trajectory(X_G, times)

Now let's plot our trajectories.  `Altair` is a plotting library built on Pandas and Vega-lite (to support interactive graphics), which works well in Colab.  I aim to start using it more.  

In [ ]:
import altair as alt

data = dataframe(traj_p_G, traj_p_G.get_segment_times(), ['x','y','z'])
alt.vconcat(
  alt.Chart(data).mark_line().encode(x='t', y='x').properties(height=80),
  alt.Chart(data).mark_line().encode(x='t', y='y').properties(height=80),
  alt.Chart(data).mark_line().encode(x='t', y='z').properties(height=80),
)


We also need to come up with a time trajectory for the orientations of our gripper.  Simple straight-line interpolation between rotations doesn't work particularly well, no matter how you are representing your rotations.  We'll use the quaternion "slerp" here (short for "spherical linear interpolation"), using the `PiecewiseQuaternionSlerp` trajectory class.

In [ ]:
from pydrake.all import PiecewiseQuaternionSlerp

def make_gripper_orientation_trajectory(X_G, times):
    """
    Constructs a gripper position trajectory from the plan "sketch".
    """
    traj = PiecewiseQuaternionSlerp();
    traj.Append(times["initial"], X_G["initial"].rotation())
    traj.Append(times["prepick"], X_G["prepick"].rotation())
    traj.Append(times["pick_start"], X_G["pick"].rotation())
    traj.Append(times["pick_end"], X_G["pick"].rotation())
    traj.Append(times["postpick"], X_G["prepick"].rotation())
    traj.Append(times["preplace"], X_G["preplace"].rotation())
    traj.Append(times["place_start"], X_G["place"].rotation())
    traj.Append(times["place_end"], X_G["place"].rotation())
    traj.Append(times["postplace"], X_G["preplace"].rotation())

    return traj

traj_R_G = make_gripper_orientation_trajectory(X_G, times)

We can plot the orientation trajectory, too.  Here are the components of the quaternion over time.

In [ ]:
data = dataframe(traj_R_G, traj_R_G.get_segment_times(), ['x','y','z','w'])
alt.vconcat(
  alt.Chart(data).mark_line().encode(x='t', y='x').properties(height=80),
  alt.Chart(data).mark_line().encode(x='t', y='y').properties(height=80),
  alt.Chart(data).mark_line().encode(x='t', y='z').properties(height=80),
  alt.Chart(data).mark_line().encode(x='t', y='w').properties(height=80)
)

Better, I think, to just plot the entire result.  Before we do, let's quickly define the finger trajectory.

In [ ]:
from pydrake.all import PiecewisePolynomial

open = np.array([0.107]);
closed = np.array([0.0]);

traj_wsg_command = PiecewisePolynomial.FirstOrderHold(
    [times["initial"], times["pick_start"]], np.hstack([[open], [open]]))
traj_wsg_command.AppendFirstOrderSegment(times["pick_end"], closed)
traj_wsg_command.AppendFirstOrderSegment(times["place_start"], closed)
traj_wsg_command.AppendFirstOrderSegment(times["place_end"], open)
traj_wsg_command.AppendFirstOrderSegment(times["postplace"], open)

In [ ]:
import altair as alt

data = dataframe(traj_wsg_command, traj_wsg_command.get_segment_times(), ['wsg'])
alt.Chart(data).mark_line().encode(x='t', y='wsg').properties(height=80)